In [60]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/macrodrigues/code/macrodrigues/skills_matcher/data/dictionaries/all_skills.csv')
df = df.drop_duplicates()
df.Label.value_counts()

TECHNICAL_SKILL    3480
GENERAL_SKILL        35
KNOWLEDGE            33
Name: Label, dtype: int64

In [57]:
df = df.sample(1000)

In [59]:
df.Label.value_counts()

TECHNICAL_SKILL    978
GENERAL_SKILL       14
KNOWLEDGE            8
Name: Label, dtype: int64

In [55]:
df = df.reset_index().drop(['index'], axis = 1)

In [56]:
l1 = []
l2 = []
for i in range(0, len(df['Skill'])):
    l1.append(df['Skill'][i])
    l2.append({'entities': [(0, len(df['Skill'][i]), df['Label'][i])]})
    
train_data = list(zip(l1, l2))

In [44]:
train_data[0:3]

[('Web platform development software',
  {'entities': [(0, 33, 'TECHNICAL_SKILL')]}),
 ('Persuasion', {'entities': [(0, 10, 'GENERAL_SKILL')]}),
 ('Spreadsheet software', {'entities': [(0, 20, 'TECHNICAL_SKILL')]})]

In [45]:
%%time
import spacy
from spacy import displacy
from spacy.tokens import DocBin
from spacy.training.example import Example
from tqdm import tqdm
import random

# set blank 'en' model 
nlp = spacy.blank('en')

# set pipeline
ner = nlp.create_pipe("ner")
nlp.add_pipe('ner')

for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])
        
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for batch in spacy.util.minibatch(train_data, size=2):
        random.shuffle(train_data)
        losses = {}
        for text, annotations in batch:
                # create Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.8)
                

CPU times: user 1min 10s, sys: 230 ms, total: 1min 10s
Wall time: 1min 10s


In [46]:
from pathlib import Path

output_dir = Path('/home/macrodrigues/code/macrodrigues/skills_matcher/data/model')
output_dir.mkdir()
nlp.to_disk(output_dir)

FileExistsError: [Errno 17] File exists: '/home/macrodrigues/code/macrodrigues/skills_matcher/data/model'

In [47]:
df = pd.read_csv('/home/macrodrigues/code/macrodrigues/skills_matcher/data/cleaned_data.csv')
df = df.dropna()

In [48]:
raw_text= df.description[200]
doc = nlp(raw_text)

ents = [(e.text, e.start, e.end, e.label_) for e in doc.ents]
#print(ents)
displacy.serve(doc, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
